In [2]:
from file_cache import *
from model.dynamic_unet import *
from torchvision  import models

In [ ]:
body = create_body(models., pretrained=True, cut=None)

In [3]:
from file_cache import *
#from dynamic_unet.base import *
import torch
from torch import nn
from fastai.vision import models
from efficientnet_pytorch import EfficientNet


encoder = nn.Sequential(*list(models.densenet121().children())[0])
unet =  models.unet.DynamicUnet(encoder, n_classes=5, img_size=(224, 224), blur=False, blur_final=False,
                    self_attention=False, y_range=None, #norm_type=NormType,
                    last_cross=True,
                    bottle=False)
print(unet(torch.rand(1,3,224,224)).shape)

torch.Size([1, 5, 224, 224])


In [2]:
from efficientnet_pytorch import EfficientNet


def eff(name='5'):
    class EfficientNet_(EfficientNet):
        def forward(self, inputs):
            """ Calls extract_features to extract features, applies final linear layer, and returns logits. """
            bs = inputs.size(0)
            # Convolution layers
            x = self.extract_features(inputs)
            return x

    return EfficientNet_.from_pretrained(f'efficientnet-b{name}', in_channels=3)


In [3]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from file_cache import *
from fastai.vision import *
from fastai.callbacks.hooks import *
from fvcore.common.registry import Registry
from torchvision import models
from enum import Enum

def flatten_moduleList(module: nn.Module)->List[nn.Module]:
    "If the ModuleList can be found in children, flatten it. Since ModuleList can not support hook "
    res_list = []
    for item in module.children():
        if isinstance(item, nn.ModuleList):
            res_list.extend(flatten_moduleList(item))
        else:
            res_list.append(item)
    return res_list


def get_unet_config(model, img_size=(512, 512)):
    "Cut the network to several blocks, the width and high of the image are reduced by half. And the image W and H >= 7"
    x = torch.rand(1, in_channels(model), *img_size)
    hooks = []
    count = 0
    layer_mata = []
    layers = []

    def hook(module, input, output):
        try:
            nonlocal count
            # print(count, type(module))
            # print(output.shape)
            if len(output.shape) == 4:
                b, c, w, h = output.shape
                layer_mata.append((count, type(module).__name__, c, w, h, output.shape))
            else:
                # pass
                print(count, output.shape, type(module))

            # channel[module.name]= output.size()[1]
            layers.append(module)
            count += 1
        except AttributeError:
            print('Error:', type(module))
            pass

    for module in flatten_moduleList(model):
        hooks.append(module.register_forward_hook(hook))

    # make a forward pass to trigger the hooks
    model(x)

    # remove these hooks
    for h in hooks:
        h.remove()

    layer_mata = pd.DataFrame(layer_mata, columns=['sn', 'layer', 'c', 'w', 'h', 'size'])
    print(layer_mata)

    img_size = [x.shape[-1] // (2 ** i) for i in range(8)]
    img_size = [size for size in img_size if size >= 7]
    layer_mata = layer_mata.loc[(layer_mata.w.isin(img_size))].drop_duplicates(['w'], keep='last')
    print(layer_mata)
    layer_sn = list(layer_mata.sn)
    layer_size = list(layer_mata['size'])
    layers = [layers[i] for i in layer_sn]
    return layer_sn, layer_size, layers



get_unet_config(eff(5))

#get_unet_config(models.resnet34())


Loaded pretrained weights for efficientnet-b5
    sn                    layer     c    w    h               size
0    0  Conv2dStaticSamePadding    48  256  256  (1, 48, 256, 256)
1    1              BatchNorm2d    48  256  256  (1, 48, 256, 256)
2    2     MemoryEfficientSwish    48  256  256  (1, 48, 256, 256)
3    3              MBConvBlock    24  256  256  (1, 24, 256, 256)
4    4              MBConvBlock    24  256  256  (1, 24, 256, 256)
5    5              MBConvBlock    24  256  256  (1, 24, 256, 256)
6    6              MBConvBlock    40  128  128  (1, 40, 128, 128)
7    7              MBConvBlock    40  128  128  (1, 40, 128, 128)
8    8              MBConvBlock    40  128  128  (1, 40, 128, 128)
9    9              MBConvBlock    40  128  128  (1, 40, 128, 128)
10  10              MBConvBlock    40  128  128  (1, 40, 128, 128)
11  11              MBConvBlock    64   64   64    (1, 64, 64, 64)
12  12              MBConvBlock    64   64   64    (1, 64, 64, 64)
13  13          

([5, 10, 15, 29, 44],
 [torch.Size([1, 24, 256, 256]),
  torch.Size([1, 40, 128, 128]),
  torch.Size([1, 64, 64, 64]),
  torch.Size([1, 176, 32, 32]),
  torch.Size([1, 2048, 16, 16])],
 [MBConvBlock(
    (_depthwise_conv): Conv2dStaticSamePadding(
      24, 24, kernel_size=(3, 3), stride=(1, 1), groups=24, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_se_reduce): Conv2dStaticSamePadding(
      24, 6, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_se_expand): Conv2dStaticSamePadding(
      6, 24, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_project_conv): Conv2dStaticSamePadding(
      24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )
    (_bn2): BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_r

In [4]:
model_sizes(eff(), (512,512))

Loaded pretrained weights for efficientnet-b5


TypeError: 'EfficientNet_' object is not iterable

In [5]:
model_sizes(nn.Sequential(*list(eff().children())), (512,512))

Loaded pretrained weights for efficientnet-b5


NotImplementedError: 

In [26]:

class Conv2dStaticSamePadding(nn.Conv2d):
    """
        It is a mini class for first layer of Efficientnet:
        https://github.com/lukemelas/EfficientNet-PyTorch/blob/master/efficientnet_pytorch/utils.py
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,  **kwargs):
        super().__init__(in_channels, out_channels, kernel_size, stride, **kwargs)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2
        self.static_padding = nn.ZeroPad2d((0, 1, 0, 1))

class dummy_efficientnet(nn.Module):
    def __init__(self):
        super().__init__()
        self._conv_head = Conv2dStaticSamePadding(3, 48, kernel_size=1)

m = nn.Sequential(*[nn.Conv2d(3,4,5)])
m = simple_cnn(b)
#m = dummy_efficientnet()

def in_channels(m:nn.Module) -> List[int]:
    "Return the shape of the first weight layer in `m`."
    for l in m.children():
        if hasattr(l, 'weight'): return l.weight.shape[1]
    raise Exception('No weight layer')
        
# def in_channels(m:nn.Module) -> List[int]:
#     "Return the shape of the first weight layer in `m`."
#     for l in flatten_model(m):
#         if hasattr(l, 'weight'): return l.weight.shape[1]
#     raise Exception('No weight layer')
    
in_channels(m)

3

In [27]:
m = simple_cnn(b)
list(m.children()) [0]

Sequential(
  (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (1): ReLU(inplace=True)
)

In [28]:
m = simple_cnn(b)
list(m.modules()) [0]

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): Conv2d(6, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (2): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten()
  )
)

In [109]:
flatten_model(SimpleTest())

[Conv2dStaticSamePadding(3, 32, kernel_size=(1, 1), stride=(1, 1))]

In [97]:

class Conv2dSamePadding(nn.Conv2d):
    """2D Convolutions with same padding
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True, name=None):
        super().__init__(in_channels, out_channels, kernel_size, stride, padding=0, dilation=dilation, groups=groups,
                         bias=bias)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2
        self.name = name

    def forward(self, x):
        input_h, input_w = x.size()[2:]
        kernel_h, kernel_w = self.weight.size()[2:]
        stride_h, stride_w = self.stride
        output_h, output_w = math.ceil(input_h / stride_h), math.ceil(input_w / stride_w)
        pad_h = max((output_h - 1) * self.stride[0] + (kernel_h - 1) * self.dilation[0] + 1 - input_h, 0)
        pad_w = max((output_w - 1) * self.stride[1] + (kernel_w - 1) * self.dilation[1] + 1 - input_w, 0)
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2])
        return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

model = nn.Sequential(*[Conv2dSamePadding(3,48,3, stride=2)])
in_channels(model)

AttributeError: 'Sequential' object has no attribute 'flatten_model'

In [89]:
eff()

Loaded pretrained weights for efficientnet-b5


EfficientNet_(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_si

In [83]:
tmp = eff()
list(tmp.children())#[2].weight.shape[1]

Loaded pretrained weights for efficientnet-b5


[Conv2dStaticSamePadding(
   3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
   (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
 ),
 BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 ModuleList(
   (0): MBConvBlock(
     (_depthwise_conv): Conv2dStaticSamePadding(
       48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
       (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
     )
     (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
     (_se_reduce): Conv2dStaticSamePadding(
       48, 12, kernel_size=(1, 1), stride=(1, 1)
       (static_padding): Identity()
     )
     (_se_expand): Conv2dStaticSamePadding(
       12, 48, kernel_size=(1, 1), stride=(1, 1)
       (static_padding): Identity()
     )
     (_project_conv): Conv2dStaticSamePadding(
       48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
       (static_padding): Identi

In [70]:
m = children_and_parameters(m)[0]
num_children(m)

1

In [72]:
children_and_parameters(m)

[ZeroPad2d(padding=(0, 1, 0, 1), value=0.0), ParameterModule()]

In [7]:
len(list(models.resnet34().children()))

10

In [39]:


flatten_model(eff(5))
        

Loaded pretrained weights for efficientnet-b5


[ZeroPad2d(padding=(0, 1, 0, 1), value=0.0),
 ParameterModule(),
 BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 ZeroPad2d(padding=(1, 1, 1, 1), value=0.0),
 ParameterModule(),
 BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 Identity(),
 ParameterModule(),
 ParameterModule(),
 Identity(),
 ParameterModule(),
 ParameterModule(),
 Identity(),
 ParameterModule(),
 BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 MemoryEfficientSwish(),
 ZeroPad2d(padding=(1, 1, 1, 1), value=0.0),
 ParameterModule(),
 BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 Identity(),
 ParameterModule(),
 ParameterModule(),
 Identity(),
 ParameterModule(),
 ParameterModule(),
 Identity(),
 ParameterModule(),
 BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 Memory

In [40]:
eff(5)

Loaded pretrained weights for efficientnet-b5


EfficientNet_(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_si

In [9]:
list(eff(5).children())

Loaded pretrained weights for efficientnet-b5


[Conv2dStaticSamePadding(
   3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
   (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
 ),
 BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 ModuleList(
   (0): MBConvBlock(
     (_depthwise_conv): Conv2dStaticSamePadding(
       48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
       (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
     )
     (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
     (_se_reduce): Conv2dStaticSamePadding(
       48, 12, kernel_size=(1, 1), stride=(1, 1)
       (static_padding): Identity()
     )
     (_se_expand): Conv2dStaticSamePadding(
       12, 48, kernel_size=(1, 1), stride=(1, 1)
       (static_padding): Identity()
     )
     (_project_conv): Conv2dStaticSamePadding(
       48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
       (static_padding): Identi

In [13]:
test = models.resnet34().cuda()

In [15]:
test2 = nn.Sequential(*list(test.children()))

In [22]:
next(test2.parameters()).device

device(type='cuda', index=0)

In [28]:
def get_unet_config( model, x=torch.rand(1, 3, 512, 512)):
    shapes = []
    blocks = OrderedDict()
    hooks = []
    count = 0
    channel = edict()
    res = []

    def register_hook(module):

        def hook(module, input, output):
            try:
                nonlocal count

                # print(output.shape)
                if len(output.shape) == 4:
                    b, c, w, h = output.shape
                    res.append((count, type(module).__name__, c, w, h))
                else:
                    pass
                    # print( count, output.shape,)

                # channel[module.name]= output.size()[1]
                count += 1
            except AttributeError:
                print('Error:', type(module))
                pass

        hooks.append(module.register_forward_hook(hook))

    # register hook
    model.apply(register_hook)

    # make a forward pass to trigger the hooks
    model(x)

    # remove these hooks
    for h in hooks:
        h.remove()

    tmp = pd.DataFrame(res, columns=['sn', 'layer', 'c', 'w', 'h'])
    return tmp
    img_size = [x.shape[-1] // (2 ** i) for i in range(6)]
    tmp = tmp.loc[(tmp.layer == 'BatchNorm2d') & (tmp.w.isin(img_size))].drop_duplicates(['w'], keep='last')

    print('layer_list, channel', list(tmp.sn), list(tmp.c))
    return list(tmp.sn), list(tmp.c)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('expand_frame_repr', False)

tmp = get_unet_config(models.resnet50())
img_size = [512 // (2 ** i) for i in range(6)]
tmp = tmp.loc[(tmp.layer == 'BatchNorm2d') & (tmp.w.isin(img_size))].drop_duplicates(['w', 'c'], keep='last')
tmp

,sn,layer,c,w,h
1,1,BatchNorm2d,64,256,256
31,31,BatchNorm2d,64,128,128
34,34,BatchNorm2d,256,128,128
39,39,BatchNorm2d,128,128,128
75,75,BatchNorm2d,128,64,64
78,78,BatchNorm2d,512,64,64
83,83,BatchNorm2d,256,64,64
139,139,BatchNorm2d,256,32,32
142,142,BatchNorm2d,1024,32,32
147,147,BatchNorm2d,512,32,32


In [53]:
from fastai.vision.models.presnet import conv_layer

In [49]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from file_cache import *
from fastai.vision import *
from fastai.callbacks.hooks import *
from fvcore.common.registry import Registry
from torchvision import models
from enum import Enum

# from ...torch_core import *
# from ...layers import *
# from ...callbacks.hooks import *

__all__ = ['DynamicUnet', 'UnetBlock']

def _get_sfs_idxs(sizes:Sizes) -> List[int]:
    "Get the indexes of the layers where the size of the activation changes."
    print('_get_sfs_idxs', sizes)
    feature_szs = [size[-1] for size in sizes]
    print('_get_sfs_idxs', sizes, feature_szs)
    sfs_idxs = list(np.where(np.array(feature_szs[:-1]) != np.array(feature_szs[1:]))[0])
    print('_get_sfs_idxs, sfs_idxs', sfs_idxs)
    print('feature_szs', feature_szs)
    if feature_szs[0] != feature_szs[1]: sfs_idxs = [0] + sfs_idxs
    print('_get_sfs_idxs, sfs_idxs', sfs_idxs)
    return sfs_idxs

def dummy_eval(m:nn.Module, size:tuple=(64,64)):
    "Pass a `dummy_batch` in evaluation mode in `m` with `size`."
    dummy = torch.rand((1,3, *size))
    print(type(m), dummy.shape)
    return m.eval()(dummy)

def model_sizes(m:nn.Module, size:tuple=(64,64))->Tuple[Sizes,Tensor,Hooks]:
    "Pass a dummy input through the model `m` to get the various sizes of activations."
    print('type(m)', type(m))
    with hook_outputs(m) as hooks:
        print('type(m)', type(m))
        x = dummy_eval(m, size)
        return [o.stored.shape for o in hooks]
    

class UnetBlock(Module):
    "A quasi-UNet block, using `PixelShuffle_ICNR upsampling`."
    def __init__(self, up_in_c:int, x_in_c:int, hook:Hook, final_div:bool=True, blur:bool=False, leaky:float=None,
                 self_attention:bool=False, **kwargs):
        self.hook = hook
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, leaky=leaky, **kwargs)
        self.bn = batchnorm_2d(x_in_c)
        ni = up_in_c//2 + x_in_c
        nf = ni if final_div else ni//2
        self.conv1 = conv_layer(ni, nf, leaky=leaky, **kwargs)
        self.conv2 = conv_layer(nf, nf, leaky=leaky, self_attention=self_attention, **kwargs)
        self.relu = relu(leaky=leaky)

    def forward(self, up_in:Tensor) -> Tensor:
        s = self.hook.stored
        up_out = self.shuf(up_in)
        ssh = s.shape[-2:]
        if ssh != up_out.shape[-2:]:
            up_out = F.interpolate(up_out, s.shape[-2:], mode='nearest')
        cat_x = self.relu(torch.cat([up_out, self.bn(s)], dim=1))
        return self.conv2(self.conv1(cat_x))


class DynamicUnet(SequentialEx):
    "Create a U-Net from a given architecture."
    def __init__(self, encoder:nn.Module, n_classes:int, img_size:Tuple[int,int]=(256,256), blur:bool=False, blur_final=True, self_attention:bool=False,
                 y_range:Optional[Tuple[float,float]]=None,
                 last_cross:bool=True, bottle:bool=False, **kwargs):
        imsize = img_size
        print('type(m)', type(encoder))
        #获取output的size
        sfs_szs = model_sizes(encoder, size=imsize)
        #获取size大小一致的最后一层的Index
        sfs_idxs = list(reversed(_get_sfs_idxs(sfs_szs)))
        print('sfs_idxs', sfs_idxs)
        #获取encode指定层的结果
        
        #print([encoder[i] for i in sfs_idxs])
        self.sfs = hook_outputs([encoder[i] for i in sfs_idxs], detach=False)
        
        #print([encoder[i] for i in sfs_idxs])
        print('self.sfs', type(self.sfs[0]), len(self.sfs), type(self.sfs))
        x = dummy_eval(encoder, imsize).detach()
        print('encode x', x.shape)
        ni = sfs_szs[-1][1]
        print('dummy_eval', x.shape , sfs_szs, ni, 'xxxx', kwargs)
        middle_conv = nn.Sequential(conv_layer(ni, ni*2, **kwargs),
                                    conv_layer(ni*2, ni, **kwargs)).eval()
        x = middle_conv(x)
        layers = [encoder, batchnorm_2d(ni), nn.ReLU(), middle_conv]
        print('sfs_idxs', sfs_idxs, x.shape)
        for i,idx in enumerate(sfs_idxs):
            print(sfs_idxs)
            not_final = i!=len(sfs_idxs)-1
            up_in_c, x_in_c = int(x.shape[1]), int(sfs_szs[idx][1])
            do_blur = blur and (not_final or blur_final)
            sa = self_attention and (i==len(sfs_idxs)-3)
            print('======', up_in_c, x_in_c,not_final,do_blur, sa, kwargs )
            #print('x.shape', x.shape)
            unet_block = UnetBlock(up_in_c, x_in_c, self.sfs[i], final_div=not_final, blur=do_blur, self_attention=sa,
                                   **kwargs).eval()
            layers.append(unet_block)
            x = unet_block(x)

        ni = x.shape[1]
        if imsize != sfs_szs[0][-2:]: layers.append(PixelShuffle_ICNR(ni, **kwargs))
        x = PixelShuffle_ICNR(ni)(x)
        print('imsize != x.shape[-2:]', imsize, x.shape[-2:], imsize != x.shape[-2:])
        if imsize != x.shape[-2:]: layers.append(Lambda(lambda x: F.interpolate(x, imsize, mode='nearest')))
        if last_cross:
            layers.append(MergeLayer(dense=True))
            ni += in_channels(encoder)
            layers.append(res_block(ni, bottle=bottle, **kwargs))
        layers += [conv_layer(ni, n_classes, ks=1, use_activ=False, **kwargs)]
        #print('y_range', y_range)
        if y_range is not None: layers.append(SigmoidRange(*y_range))
        super().__init__(*layers)

    def __del__(self):
        if hasattr(self, "sfs"): self.sfs.remove()


            
            
            
# NormType = Enum('NormType', 'Batch BatchZero Weight Spectral')
# def unet_dynamic(arch: Callable = models.resnet50, pretrained: bool = True, blur_final: bool = True,
#                  norm_type: Optional[NormType] = NormType, split_on: Optional[SplitFuncOrIdxList] = None,
#                  blur: bool = False,
#                  self_attention: bool = False, y_range: Optional[Tuple[float, float]] = None, last_cross: bool = True,
#                  bottle: bool = False, cut: Union[int, Callable] = None,
#                  n_classes=2, img_size=(224, 224), in_channels=1,
#                  **learn_kwargs: Any) -> Learner:
from efficientnet_pytorch import EfficientNet
# def eff(*args, **kwargs):
#      return EfficientNet.from_pretrained(f'efficientnet-b0', in_channels=3)

body = models.resnet50()

body = nn.Sequential(*list(models.resnet34().children())[:-2])

#print(body)
import ipdb
#ipdb.set_trace()
model = to_device(

   DynamicUnet(body, n_classes=5, img_size=(244,244), blur=False, blur_final=False,
                            self_attention=False, y_range=None, norm_type=NormType,
                            last_cross=True,
                            bottle=False), 'cuda')


model(torch.rand(1,3,224,224).cuda()) 
 
model

type(m) <class 'torch.nn.modules.container.Sequential'>
type(m) <class 'torch.nn.modules.container.Sequential'>
type(m) <class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'> torch.Size([1, 3, 244, 244])
_get_sfs_idxs [torch.Size([1, 64, 122, 122]), torch.Size([1, 64, 122, 122]), torch.Size([1, 64, 122, 122]), torch.Size([1, 64, 61, 61]), torch.Size([1, 64, 61, 61]), torch.Size([1, 128, 31, 31]), torch.Size([1, 256, 16, 16]), torch.Size([1, 512, 8, 8])]
_get_sfs_idxs [torch.Size([1, 64, 122, 122]), torch.Size([1, 64, 122, 122]), torch.Size([1, 64, 122, 122]), torch.Size([1, 64, 61, 61]), torch.Size([1, 64, 61, 61]), torch.Size([1, 128, 31, 31]), torch.Size([1, 256, 16, 16]), torch.Size([1, 512, 8, 8])] [122, 122, 122, 61, 61, 31, 16, 8]
_get_sfs_idxs, sfs_idxs [2, 4, 5, 6]
feature_szs [122, 122, 122, 61, 61, 31, 16, 8]
_get_sfs_idxs, sfs_idxs [2, 4, 5, 6]
sfs_idxs [6, 5, 4, 2]
self.sfs <class 'fastai.callbacks.hooks.Hook'> 4 <class 'fastai.callb

DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05

In [61]:
def unet_fsai_r34( pretrained: bool = True, blur_final: bool = True,
                 norm_type: Optional[NormType] = NormType, split_on: Optional[SplitFuncOrIdxList] = None,
                 blur: bool = False,
                 self_attention: bool = False, y_range: Optional[Tuple[float, float]] = None, last_cross: bool = True,
                 bottle: bool = False, cut: Union[int, Callable] = None,
                 n_classes=2, img_size=(224, 224), in_channels=1,
                 **learn_kwargs: Any) -> Learner:
    "Build Unet learner from `data` and `arch`."
    "blur: do maxpolling or not"
    from fastai.vision import models
    arch: Callable = models.resnet34
    body = create_body(arch, pretrained, cut)
    from fastai.vision import models
    model = to_device(
        DynamicUnet(body, n_classes=n_classes, img_size=img_size, blur=blur, blur_final=blur_final,
                                self_attention=self_attention, y_range=y_range, norm_type=norm_type,
                                last_cross=last_cross,
                                bottle=bottle), 'cuda')
    return model
unet_fsai_r34(img_size=(224, 224))

type(m) <class 'torch.nn.modules.container.Sequential'>
type(m) <class 'torch.nn.modules.container.Sequential'>
type(m) <class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'> torch.Size([1, 3, 224, 224])
_get_sfs_idxs [torch.Size([1, 64, 112, 112]), torch.Size([1, 64, 112, 112]), torch.Size([1, 64, 112, 112]), torch.Size([1, 64, 56, 56]), torch.Size([1, 64, 56, 56]), torch.Size([1, 128, 28, 28]), torch.Size([1, 256, 14, 14]), torch.Size([1, 512, 7, 7])]
_get_sfs_idxs [torch.Size([1, 64, 112, 112]), torch.Size([1, 64, 112, 112]), torch.Size([1, 64, 112, 112]), torch.Size([1, 64, 56, 56]), torch.Size([1, 64, 56, 56]), torch.Size([1, 128, 28, 28]), torch.Size([1, 256, 14, 14]), torch.Size([1, 512, 7, 7])] [112, 112, 112, 56, 56, 28, 14, 7]
_get_sfs_idxs, sfs_idxs [2, 4, 5, 6]
feature_szs [112, 112, 112, 56, 56, 28, 14, 7]
_get_sfs_idxs, sfs_idxs [2, 4, 5, 6]
sfs_idxs [6, 5, 4, 2]
self.sfs <class 'fastai.callbacks.hooks.Hook'> 4 <class 'fastai.callb

DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05

In [58]:
x=torch.rand(1,30,224,224)
print(x.sum())
z=F.interpolate(x, [224,224], mode='nearest')
print(z.sum())

tensor(752542.1875)
tensor(752542.1875)


In [142]:
a = ['a', 'b',1,2,3,4,'d','e']
[a[item] for item in [1,2,3,4]]

['b', 1, 2, 3]

In [ ]:
models.resnet50().apply(print(type))

In [88]:
tmp = eff() 
[ type(layer) for layer in list(tmp.modules())]

Loaded pretrained weights for efficientnet-b0


[efficientnet_pytorch.model.EfficientNet,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 torch.nn.modules.padding.ZeroPad2d,
 torch.nn.modules.batchnorm.BatchNorm2d,
 torch.nn.modules.container.ModuleList,
 efficientnet_pytorch.model.MBConvBlock,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 torch.nn.modules.padding.ZeroPad2d,
 torch.nn.modules.batchnorm.BatchNorm2d,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 efficientnet_pytorch.utils.Identity,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 efficientnet_pytorch.utils.Identity,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 efficientnet_pytorch.utils.Identity,
 torch.nn.modules.batchnorm.BatchNorm2d,
 efficientnet_pytorch.utils.MemoryEfficientSwish,
 efficientnet_pytorch.model.MBConvBlock,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 efficientnet_pytorch.utils.Identity,
 torch.nn.modules.batchnorm.BatchNorm2d,
 efficientnet_pytorch.utils.Conv2dStaticSamePadding,
 torch.nn.modules.padding

In [91]:
tmp

,sn,layer,c,w,h
0,0,ZeroPad2d,3,513,513
1,1,Conv2dStaticSamePadding,32,256,256
2,2,BatchNorm2d,32,256,256
3,3,MemoryEfficientSwish,32,256,256
4,4,ZeroPad2d,32,258,258
...,...,...,...,...,...
272,272,Identity,320,16,16
273,273,Conv2dStaticSamePadding,1280,16,16
274,274,BatchNorm2d,1280,16,16
275,275,MemoryEfficientSwish,1280,16,16
